In [1]:
import pulsenet as pn
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models

2023-05-12 14:13:59.996682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 14:14:01.548352: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-12 14:14:01.548588: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-12 14:14:04.229136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

In [6]:
################# modelfiles #######################
# direct transfer
transfer_model = "models/transfer_model.h5"
# direct transfer with fine tuning
fine_tuned_model = "models/fine_tuned_model.h5"
# long transfer
transfer_model_trace_transfer = "models/transfer_trace_transfer_model.h5"
# long transfer with fine tuning
fine_tuned_trace_transfer_model = "models/fine_tuned_trace_transfer_model.h5"
# control
control_model = "models/control_model.h5"
# sequential
sequential_model ="models/sequential_training_model.h5"

input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "transfer_head")([discriminator_feature_vec, classifer_feature_vec])
phase_output = pn.TracePhaseRegressor(name="phase_regressor")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude_regressor")(trace_output)
classifier_output = pn.TraceClassifierHead(name="classifier_head")(classifer_feature_vec)

control = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
control.load_weights(control_model)

transfer_direct = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
transfer_direct.load_weights(fine_tuned_model)



In [10]:
filename = "ysoTracesNoPileup.root"
x_trace, y_trace, y_phase, y_amp =  pn.CreateData(filename, pileup_split=0.5, phase_min=0, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
y_pileup_onehot = pn.EncodePileup(y_phase)   

Created 437552 samples: 49.96594690459648 % pileup, 50.03405309540352 % no pileup


In [12]:
# predict on data
control_trace_pred, control_phase_pred, control_amp_pred = control.predict(x_trace)
transfer_trace_pred, transfer_phase_pred, transfer_amp_pred = transfer_direct.predict(x_trace)

2023-05-12 14:27:53.400600: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 525062400 exceeds 10% of free system memory.


13674/13674 [==============================] - 349s 25ms/step


2023-05-12 14:33:58.482199: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1050124800 exceeds 10% of free system memory.


KeyboardInterrupt: 

In [ ]:
# evaluate the model
control_loss = control.evaluate(x_trace, [y_trace, y_phase, y_amp])
transfer_loss = transfer_direct.evaluate(x_trace, [y_trace, y_phase, y_amp])


In [ ]:
# calulate residuals
phase_fine_tuned_residuals = phase_pred_fine_tuned - y_phase
phase_transfer_learned_residuals = phase_pred_transfer_learned - y_phase
phase_initial_residuals = phase_pred_initial - y_phase

amp_fine_tuned_residuals = amp_pred_fine_tuned - y_amp
amp_transfer_learned_residuals = amp_pred_transfer_learned - y_amp
amp_initial_residuals = amp_pred_initial - y_amp

In [ ]:
from sklearn.metrics import roc_curve
pred_class_fine_tuned = pileup_pred_fine_tuned[:,0] # pileup classifier output is (1,0) if pileup, (0,1) if not pileup
pred_class_transfer_learned = pileup_pred_transfer_learned[:,0]
pred_class_initial = pileup_pred_initial[:,0]
truth_class = y_pileup_onehot[:,0] # one hot encoded truth is (1,0) if pileup, (0,1) if not pileup

ft_fpr, ft_tpr, ft_thresholds = roc_curve(truth_class, pred_class_fine_tuned)
tl_fpr, tl_tpr, tl_thresholds = roc_curve(truth_class, pred_class_transfer_learned)
in_fpr, in_tpr, in_thresholds = roc_curve(truth_class, pred_class_initial)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ft_fpr, ft_tpr, label="Fine Tuned")
plt.plot(tl_fpr, tl_tpr, label="Transfer Learned")
plt.plot(in_fpr, in_tpr, label="Initial")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label="No Skill")
plt.legend()

In [ ]:
import numpy as np
import pandas as pd
phase_bins = np.linspace(0,20,20)
amp_bins = np.linspace(0.5,1.5,20)

pdf = pd.DataFrame({"Phase":y_phase[:,0], "Phase Fine Tuned":phase_pred_fine_tuned[:,0], "Phase Transfer Learned":phase_pred_transfer_learned[:,0], "Phase Initial":phase_pred_initial[:,0]})
adf = pd.DataFrame({"Amplitude":y_amp[:,0], "Amplitude Fine Tuned":amp_pred_fine_tuned[:,0], "Amplitude Transfer Learned":amp_pred_transfer_learned[:,0], "Amplitude Initial":amp_pred_initial[:,0]})

p_ft_sigma = []
p_tl_sigma = []
p_in_sigma = []
a_ft_sigma = []
a_tl_sigma = []
a_in_sigma = []


for i in range(0,19):
    pdf_bin = pdf[(pdf["Phase"] > phase_bins[i]) & (pdf["Phase"] < phase_bins[i+1])].copy()
    adf_bin = adf[(adf["Amplitude"] > amp_bins[i]) & (adf["Amplitude"] < amp_bins[i+1])].copy()
    p_ft_sigma.append(pdf_bin["Phase Fine Tuned"].std())
    p_tl_sigma.append(pdf_bin["Phase Transfer Learned"].std())
    p_in_sigma.append(pdf_bin["Phase Initial"].std())
    a_ft_sigma.append(adf_bin["Amplitude Fine Tuned"].std())
    a_tl_sigma.append(adf_bin["Amplitude Transfer Learned"].std())
    a_in_sigma.append(adf_bin["Amplitude Initial"].std())   


In [ ]:
plt.plot(phase_bins[:-1], p_ft_sigma, label="Fine Tuned")
plt.plot(phase_bins[:-1], p_tl_sigma, label="Transfer Learned")
plt.plot(phase_bins[:-1], p_in_sigma, label="Initial")
plt.xlabel(r"Phase $\phi$ (ns)")
plt.ylabel(r"$\sigma(\delta\phi)$")
plt.legend()

In [ ]:
plt.plot(amp_bins[:-1], a_ft_sigma, label="Fine Tuned")
plt.plot(amp_bins[:-1], a_tl_sigma, label="Transfer Learned")
plt.plot(amp_bins[:-1], a_in_sigma, label="Initial")
plt.xlabel(r"Amplitude $A$ (mV)")
plt.ylabel(r"$\sigma(\delta A)$")
plt.legend()

In [ ]:
def KL_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

ft_kl0 = KL_divergence(trace_pred_fine_tuned[:,0], y_trace[:,0])
tl_kl0= KL_divergence(trace_pred_transfer_learned[:,0], y_trace[:,0])   
in_kl0 = KL_divergence(trace_pred_initial[:,0], y_trace[:,0])

ft_kl1 = KL_divergence(phase_pred_fine_tuned[:,0], y_phase[:,0])
tl_kl1 = KL_divergence(phase_pred_transfer_learned[:,0], y_phase[:,0])
in_kl1 = KL_divergence(phase_pred_initial[:,0], y_phase[:,0])

ft_kl = np.concatenate((ft_kl0, ft_kl1), axis=0)
tl_kl = np.concatenate((tl_kl0, tl_kl1), axis=0)
in_kl = np.concatenate((in_kl0, in_kl1), axis=0)

In [ ]:
plt.hist(ft_kl, bins=100, label="Fine Tuned", alpha=0.5, density=True)
plt.hist(tl_kl, bins=100, label="Transfer Learned", alpha=0.5, density=True)
plt.hist(in_kl, bins=100, label="Initial", alpha=0.5, density=True)
plt.xlabel("KL Divergence")
plt.ylabel("Count")
plt.legend()